## Pandas API in Apache Beam

Apache Beam 2.26 onwards supports the Pandas API. This makes it very convenient to write complex pipelines, and execute them at scale, or in a streaming manner.

In [1]:
%pip install --quiet apache-beam[gcp]==2.26.0

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tfx 0.23.0 requires attrs<20,>=19.3.0, but you'll have attrs 20.3.0 which is incompatible.
tfx 0.23.0 requires google-resumable-media<0.7.0,>=0.6.0, but you'll have google-resumable-media 1.1.0 which is incompatible.
tfx 0.23.0 requires kubernetes<12,>=10.0.1, but you'll have kubernetes 12.0.0 which is incompatible.
tensorflow-data-validation 0.23.1 requires joblib<0.15,>=0.12, but you'll have joblib 0.17.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import apache_beam as beam
import pandas as pd
print(beam.__version__)

2.26.0


## 1. Experiment on one day of flights data

Let's pull out one day of data using BigQuery and display the stats we need.

In [3]:
%%bigquery df
SELECT  
  airline,
  departure_airport,
  arrival_airport,
  departure_delay,
  arrival_delay
FROM `bigquery-samples.airline_ontime_data.flights`
WHERE date = '2006-08-20'

In [4]:
df.head()

,airline,departure_airport,arrival_airport,departure_delay,arrival_delay
0,AS,ANC,DEN,-1.0,-12.0
1,F9,ANC,DEN,0.0,0.0
2,UA,ANC,DEN,-7.0,14.0
3,F9,PHX,DEN,14.0,8.0
4,F9,PHX,DEN,-2.0,4.0


In [5]:
# most frequent airports used by each carrier
aa = df.groupby('airline').get_group('AA')

arr = aa.rename(columns={'arrival_airport': 'airport'}).airport.value_counts()
arr.head()

DFW    447
ORD    214
MIA    112
LAX     86
LGA     59
Name: airport, dtype: int64

In [6]:
dep = aa.rename(columns={'departure_airport': 'airport'}).airport.value_counts()
dep.head()

DFW    450
ORD    213
MIA    112
LAX     86
LGA     57
Name: airport, dtype: int64

In [7]:
total = arr + dep
top_airports = total.nlargest(10)
top_airports.index.values

array(['DFW', 'ORD', 'MIA', 'LAX', 'LGA', 'STL', 'SJU', 'BOS', 'SFO',
       'JFK'], dtype=object)

In [8]:
means = aa[aa['arrival_airport'].isin(top_airports.index.values)].mean()
means

departure_delay    4.533036
arrival_delay      1.350000
dtype: float64

In [9]:
print({
    'airline': aa.airline.iloc[0],
    'departure_delay': means['departure_delay'],
    'arrival_delay': means['arrival_delay'],
})

{'airline': 'AA', 'departure_delay': 4.533035714285714, 'arrival_delay': 1.35}


## 2. Combine the Pandas code into functions

Make functions out of the Pandas code so that it is repeatable

In [10]:
# do this for all the carriers
def get_delay_at_top_airports(aa):
    arr = aa.rename(columns={'arrival_airport': 'airport'}).airport.value_counts()
    dep = aa.rename(columns={'departure_airport': 'airport'}).airport.value_counts()
    total = arr + dep
    top_airports = total.nlargest(10)
    means = aa[aa['arrival_airport'].isin(top_airports.index.values)].mean()
    return '{:2f},{:2f}'.format(
        means['departure_delay'], means['arrival_delay'])

df.groupby('airline').apply(get_delay_at_top_airports)

airline
AA      4.533036,1.350000
AS    21.035503,22.032544
B6      6.363905,5.890533
CO      7.312169,2.442681
DL    13.549398,17.514458
EV    22.855721,26.079602
F9      3.686170,3.882979
FL    20.049217,23.006711
HA    -3.890411,-3.383562
KH    -1.608392,-1.552448
MQ      5.169554,3.707921
NW      4.281379,2.769655
OH      4.923445,6.473684
OO      9.402878,8.790168
TZ     9.333333,11.588235
UA      6.776151,3.776151
US      4.423826,1.066438
WN     4.107368,-2.775439
XE      3.317007,2.081633
YV    15.212632,10.185263
dtype: object

## 3. Productionize pipeline using Apache Beam on Dataflow

Apache Beam lets you run batch and streaming pipelines at scale and in resilient way.
To do this, build a pipeline.
* Do it on full dataset (batch)
* Do it on streaming data by adding a Sliding or Fixed Time Window to process daily/hourly/minute-by-minute data as it comes in.

In [11]:
import time
import datetime
def to_unixtime(s):
    return time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple())
print(to_unixtime('2006-08-20'))

1156032000.0


In [15]:
from apache_beam.dataframe.convert import to_dataframe, to_pcollection

In [20]:
query = """
SELECT  
  date,
  airline,
  departure_airport,
  arrival_airport,
  departure_delay,
  arrival_delay
FROM `bigquery-samples.airline_ontime_data.flights`
"""
with beam.Pipeline() as p:
    tbl = (p 
           | 'read table' >> beam.io.ReadFromBigQuery(query=query)
           | 'assign ts' >> beam.Map(
               lambda x: beam.window.TimestampedValue(x, to_unixtime(x['date'])))
           | 'set schema' >> beam.Select(
               date=lambda x: str(x['date']),
               airline=lambda x: str(x['airline']),
               departure_airport=lambda x: str(x['departure_airport']),
               arrival_airport=lambda x: str(x['arrival_airport']),
               departure_delay=lambda x: float(x['departure_delay']),
               arrival_delay=lambda x: float(x['arrival_delay']))
          )
    daily = tbl | 'daily windows' >> beam.WindowInto(beam.window.FixedWindows(60*60*24))
    # group the flights data by carrier
    df = to_dataframe(daily)
    grouped = df.groupby('airline')
    #agg = to_pcollection(grouped.groups)
    # get dataframes corresponding to each group, and apply our function to it
    #result = agg | 'avg delays' >> beam.Map(lambda pc: to_dataframe(pc).get_delay_at_top_airports)
    result = grouped.apply(get_delay_at_top_airports)
    result.to_csv('output.csv')

NotImplementedError: 'apply' is not yet supported (BEAM-9547)

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.